### Load libraries

In [1]:
#load libraries
import pandas as pd
import numpy as np
import matplotlib as plt
import os
import seaborn as sns
import geopandas as gpd

### Read data

In [2]:
#read the ICER data
icer = pd.read_excel("../data/raw/ICER_data.xlsx")

In [3]:
#read in the data for life expectancy
le_district = pd.read_csv("../data/processed/le_combined.csv")
le_nat = pd.read_csv("../data/processed/le_calc_national.csv")
df_map = gpd.read_file("../data/processed/high_res.shp")

In [4]:
#rename first column as treatment
icer = icer.rename(columns={"Unnamed: 0": "Treatment"})

In [5]:
icer.head()

,Treatment,burden,qaly_lost,healthcare_cost,payer_cost
0,non_vaccine,382416,184088.54,26618814,16066984
1,vaccine,264217,127189.68,352828922,345497041


### Calculate district-level ICER based on the available ICER (average) from 2020-2024 in Indonesia (from research paper), nationwide, adjusted by life expectancy to obtain district-level ICER, assuming intervention costs remain approximately the same (similar) --> life expectancy is to adjust based on QALYs. All prices are adjusted to 2018 USD based on PPP and discounted by 3%

In [6]:
le_district.head()

,District,Year,Life Expectancy
0,ACEH,2022,70.18
1,SIMEULUE,2022,65.48
2,ACEH SINGKIL,2022,67.65
3,ACEH SELATAN,2022,64.64
4,ACEH TENGGARA,2022,68.48


In [7]:
#rename as district_life_exp
le_district = le_district.rename(columns={"Life Expectancy": "district_le"})

In [8]:
le_nat.head()

,Year,Life Expectancy
0,2019,69.213148
1,2020,69.363809
2,2021,69.474243
3,2022,69.747513
4,2023,70.018191


In [9]:
#rename as nat_le
le_nat = le_nat.rename(columns={"Life Expectancy": "nat_le"})

In [10]:
#merge le_district and le_nat by year
le_final = pd.merge(le_district, le_nat, on = "Year")

#replace all the NA values

#make sure all the life expectancy values are all floats
le_final["district_le"] = le_final["district_le"].astype(float)
le_final["nat_le"] = le_final["nat_le"].astype(float)

#calculate the le_factor, which is a comparison of the district-level to the national-level life expectancy
le_final["le_factor"] = le_final["district_le"] / le_final["nat_le"]

In [11]:
#create another column in df_map to calculate the discounting factor for each district in dki jakarta and west java
df = pd.merge(df_map, le_final, on = ("District", "Year"))

In [12]:
#remove the district_le and nat_le columns
df_final = df.drop(columns = ["district_le", "nat_le"])

In [13]:
df_final["healthcare_cost_non_vaccine"] = icer["healthcare_cost"].iloc[0]
df_final["healthcare_cost_vaccine"] = icer["healthcare_cost"].iloc[1]
df_final["payer_cost_non_vaccine"] = icer["payer_cost"].iloc[0]
df_final["payer_cost_vaccine"] = icer["payer_cost"].iloc[1]
df_final["qaly_lost_non_vaccine"] = icer["qaly_lost"].iloc[0]
df_final["qaly_lost_vaccine"] = icer["qaly_lost"].iloc[1]
df_final["burden_non_vaccine"] = icer["burden"].iloc[0]
df_final["burden_vaccine"] = icer["burden"].iloc[1]

In [14]:
# Scale total cost to produce district-level cost per year based on cases per district per year / total num of cases over 2020-2024 nationwide, assuming that healthcare costs for dengue treatment is linearly related to the number of dengue cases per year
# this cost factor measures the scale of the district-level, yearly dengue cases to the national-level dengue cases in total from 2020-2024.
df_final['cost_factor'] = df_final['Cases'] / (df_final['burden_non_vaccine'] + df_final['burden_vaccine'])


In [15]:
#calculate district-level adjusted ICER for payers and healthcare
# Scale QALYs lost using le_factor and the cost-factor to disaggregate the qalys to district-level, yearly measurements from the national-level total qalys lost from 2020-2024
df_final['qaly_le_scaled_vaccine'] = df_final['qaly_lost_vaccine'] * df_final['le_factor']
df_final['qaly_le_scaled_nonvaccine'] = df_final['qaly_lost_non_vaccine'] * df_final['le_factor']

df_final['qaly_lost_scaled_vaccine'] = df_final['qaly_le_scaled_vaccine'] * df_final['cost_factor']
df_final['qaly_lost_scaled_nonvaccine'] = df_final['qaly_le_scaled_nonvaccine'] * df_final['cost_factor']

In [16]:
# Scale costs
df_final['healthcare_cost_vaccine_scaled'] = df_final['healthcare_cost_vaccine'] * df_final['cost_factor']
df_final['healthcare_cost_nonvaccine_scaled'] = df_final['healthcare_cost_non_vaccine'] * df_final['cost_factor']
df_final['payer_cost_vaccine_scaled'] = df_final['payer_cost_vaccine'] * df_final['cost_factor']
df_final['payer_cost_nonvaccine_scaled'] = df_final['payer_cost_non_vaccine'] * df_final['cost_factor']

df_final.head()

,District,x,y,Year,Area_sq_km,HDI,Province,Cases,Population,Pop_den,...,burden_vaccine,cost_factor,qaly_le_scaled_vaccine,qaly_le_scaled_nonvaccine,qaly_lost_scaled_vaccine,qaly_lost_scaled_nonvaccine,healthcare_cost_vaccine_scaled,healthcare_cost_nonvaccine_scaled,payer_cost_vaccine_scaled,payer_cost_nonvaccine_scaled
0,BANDUNG,107.610841,-7.099969,2020,1767.96,72.39,Jawa Barat,9180,14495160,8198.805403,...,264217,0.014197,134829.060662,195145.431036,1914.116318,2770.404630,5.008977e+06,377897.064517,4.904889e+06,228096.792338
1,BANDUNG,107.610841,-7.099969,2021,1767.96,72.73,Jawa Barat,8008,14662620,8293.524740,...,264217,0.012384,134962.580953,195338.682213,1671.396833,2419.103521,4.369486e+06,329651.382642,4.278687e+06,198975.938240
2,BANDUNG,107.610841,-7.099969,2022,1767.96,73.16,Jawa Barat,16764,14830092,8388.250865,...,264217,0.025925,134962.636029,195338.761927,3498.914578,5064.169328,9.147111e+06,690094.377949,8.957032e+06,416537.541041
3,BANDUNG,107.610841,-7.099969,2023,1767.96,73.74,Jawa Barat,4020,14997564,8482.976990,...,264217,0.006217,134913.189810,195267.195726,838.730815,1213.940716,2.193473e+06,165484.335442,2.147892e+06,99885.523442
4,BANDUNG BARAT,107.414953,-6.897056,2020,1305.77,68.08,Jawa Barat,3864,7153344,5478.257273,...,264217,0.005976,132647.004601,191987.222646,792.641306,1147.232864,2.108354e+06,159062.555261,2.064541e+06,96009.368801


In [17]:
# Compute incremental costs
df_final['incremental_healthcare_cost'] = df_final['healthcare_cost_vaccine_scaled'] - df_final['healthcare_cost_nonvaccine_scaled']
df_final['incremental_payer_cost'] = df_final['payer_cost_vaccine_scaled'] - df_final['payer_cost_nonvaccine_scaled']

df_final.head()

,District,x,y,Year,Area_sq_km,HDI,Province,Cases,Population,Pop_den,...,qaly_le_scaled_vaccine,qaly_le_scaled_nonvaccine,qaly_lost_scaled_vaccine,qaly_lost_scaled_nonvaccine,healthcare_cost_vaccine_scaled,healthcare_cost_nonvaccine_scaled,payer_cost_vaccine_scaled,payer_cost_nonvaccine_scaled,incremental_healthcare_cost,incremental_payer_cost
0,BANDUNG,107.610841,-7.099969,2020,1767.96,72.39,Jawa Barat,9180,14495160,8198.805403,...,134829.060662,195145.431036,1914.116318,2770.404630,5.008977e+06,377897.064517,4.904889e+06,228096.792338,4.631079e+06,4.676792e+06
1,BANDUNG,107.610841,-7.099969,2021,1767.96,72.73,Jawa Barat,8008,14662620,8293.524740,...,134962.580953,195338.682213,1671.396833,2419.103521,4.369486e+06,329651.382642,4.278687e+06,198975.938240,4.039835e+06,4.079711e+06
2,BANDUNG,107.610841,-7.099969,2022,1767.96,73.16,Jawa Barat,16764,14830092,8388.250865,...,134962.636029,195338.761927,3498.914578,5064.169328,9.147111e+06,690094.377949,8.957032e+06,416537.541041,8.457017e+06,8.540494e+06
3,BANDUNG,107.610841,-7.099969,2023,1767.96,73.74,Jawa Barat,4020,14997564,8482.976990,...,134913.189810,195267.195726,838.730815,1213.940716,2.193473e+06,165484.335442,2.147892e+06,99885.523442,2.027989e+06,2.048007e+06
4,BANDUNG BARAT,107.414953,-6.897056,2020,1305.77,68.08,Jawa Barat,3864,7153344,5478.257273,...,132647.004601,191987.222646,792.641306,1147.232864,2.108354e+06,159062.555261,2.064541e+06,96009.368801,1.949291e+06,1.968532e+06


In [18]:
# Compute incremental QALY
df_final['incremental_qaly'] = (df_final['qaly_lost_scaled_nonvaccine'] - df_final['qaly_lost_scaled_vaccine'])

In [19]:
# Compute ICER correctly
df_final['icer_healthcare_observed'] = abs(df_final['incremental_healthcare_cost'] / df_final['incremental_qaly'])
df_final['icer_payer_observed'] = abs(df_final['incremental_payer_cost'] / df_final['incremental_qaly'])

df_final.head()

,District,x,y,Year,Area_sq_km,HDI,Province,Cases,Population,Pop_den,...,qaly_lost_scaled_nonvaccine,healthcare_cost_vaccine_scaled,healthcare_cost_nonvaccine_scaled,payer_cost_vaccine_scaled,payer_cost_nonvaccine_scaled,incremental_healthcare_cost,incremental_payer_cost,incremental_qaly,icer_healthcare_observed,icer_payer_observed
0,BANDUNG,107.610841,-7.099969,2020,1767.96,72.39,Jawa Barat,9180,14495160,8198.805403,...,2770.404630,5.008977e+06,377897.064517,4.904889e+06,228096.792338,4.631079e+06,4.676792e+06,856.288312,5408.317941,5461.702270
1,BANDUNG,107.610841,-7.099969,2021,1767.96,72.73,Jawa Barat,8008,14662620,8293.524740,...,2419.103521,4.369486e+06,329651.382642,4.278687e+06,198975.938240,4.039835e+06,4.079711e+06,747.706688,5402.967419,5456.298935
2,BANDUNG,107.610841,-7.099969,2022,1767.96,73.16,Jawa Barat,16764,14830092,8388.250865,...,5064.169328,9.147111e+06,690094.377949,8.957032e+06,416537.541041,8.457017e+06,8.540494e+06,1565.254750,5402.965214,5456.296708
3,BANDUNG,107.610841,-7.099969,2023,1767.96,73.74,Jawa Barat,4020,14997564,8482.976990,...,1213.940716,2.193473e+06,165484.335442,2.147892e+06,99885.523442,2.027989e+06,2.048007e+06,375.209901,5404.945422,5458.296463
4,BANDUNG BARAT,107.414953,-6.897056,2020,1305.77,68.08,Jawa Barat,3864,7153344,5478.257273,...,1147.232864,2.108354e+06,159062.555261,2.064541e+06,96009.368801,1.949291e+06,1.968532e+06,354.591557,5497.285294,5551.547801


In [20]:
#only retain the necessary rows and columns, save it as the benchmark ICER calculations
df_final = df_final[["District", "Province", "Population", "Year", "icer_healthcare_observed", "icer_payer_observed", "qaly_le_scaled_vaccine", "qaly_le_scaled_nonvaccine", "burden_non_vaccine", 'burden_vaccine', 'healthcare_cost_non_vaccine', 'healthcare_cost_vaccine', 'payer_cost_non_vaccine', 'payer_cost_vaccine']]

### Save the df for other icer calculations

In [21]:
df_final.to_csv("../data/processed/icer_observed.csv")